In [21]:
import os
import pandas as pd
import xgboost
import utils
import scoring
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
DATA_PATH = "./data"

In [4]:
train, test = utils.load_data_csv(
    DATA_PATH, utils.SIMPLE_FEATURE_COLUMNS)

/Users/aramdavtyan/datascience-env/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [18]:
train = train.fillna(train.mean(0))

In [19]:
test = test.fillna(test.mean(0))

In [20]:
train_part, validation = train_test_split(train, test_size=0.25, shuffle=True)

In [26]:
model = xgboost.XGBClassifier(n_estimators=600, n_jobs=-1)

In [27]:
model.fit(train_part.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values,
          train_part.label.values,
          sample_weight=train_part.weight.values)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=600,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [28]:
validation_predictions = model.predict_proba(validation.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values)[:, 1]

In [29]:
scoring.rejection90(validation.label.values, validation_predictions, sample_weight=validation.weight.values)

0.5906645763733432

In [12]:
model.fit(train.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values, train.label, sample_weight=train.weight)

[18:53:50] Tree method is automatically selected to be 'approx' for faster speed. to use old behavior(exact greedy algorithm on single machine), set tree_method to 'exact'


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [13]:
predictions = model.predict_proba(test.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values)[:, 1]

In [14]:
pd.DataFrame(data={"prediction": predictions}, index=test.index).to_csv(
    "sample_submission.csv", index_label=utils.ID_COLUMN)

In [18]:
model_track_2 = xgboost.XGBClassifier(n_estimators=150, n_jobs=-1).fit(
    train.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values, train.label, sample_weight=train.weight)

[19:26:52] Tree method is automatically selected to be 'approx' for faster speed. to use old behavior(exact greedy algorithm on single machine), set tree_method to 'exact'


In [19]:
model_track_2.save_model("track_2_baseline_simple_python/track_2_model.xgb")